In [171]:
# Importing the required packages
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import lower, col
from pyspark.sql import functions as sf

In [172]:
spark = SparkSession.builder.getOrCreate()

# Checking if spark session is working
df = spark.sql('''select 'spark' as hello ''')
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [173]:
sc = spark.sparkContext

In [174]:
#Importing raw data into a Dataframe
df_whitehouse = spark.read.csv("whitehouse_waves-2016_12.csv", header=True, inferSchema=True)

In [175]:
df_whitehouse.show(2)

+---------------+---------+-------+------+------+-----------+----+----+----+----+--------------+---------------+---------------+----------------+------------+--------------+----+---------------+---------------+----------------+-----------------+-----------+------------+----------------+-----------------+-----------+-----------+------------+
|       NAMELAST|NAMEFIRST|NAMEMID|   UIN|BDGNBR|ACCESS_TYPE| TOA| POA| TOD| POD|APPT_MADE_DATE|APPT_START_DATE|  APPT_END_DATE|APPT_CANCEL_DATE|Total_People|LAST_UPDATEDBY|POST|  LASTENTRYDATE|TERMINAL_SUFFIX|visitee_namelast|visitee_namefirst|MEETING_LOC|MEETING_ROOM|CALLER_NAME_LAST|CALLER_NAME_FIRST|CALLER_ROOM|DESCRIPTION|Release_Date|
+---------------+---------+-------+------+------+-----------+----+----+----+----+--------------+---------------+---------------+----------------+------------+--------------+----+---------------+---------------+----------------+-----------------+-----------+------------+----------------+-----------------+---------

In [176]:
#Required Columns - NAMEFIRST, NAMELAST, visitee_namefirst, visitee_namelast
df_req = df_whitehouse.select('NAMEFIRST', 'NAMELAST', 'visitee_namefirst', 'visitee_namelast')
df_req.show(2)

+---------+---------------+-----------------+----------------+
|NAMEFIRST|       NAMELAST|visitee_namefirst|visitee_namelast|
+---------+---------------+-----------------+----------------+
|   HANENE|TAJOURIBESSASSI|             Eric|        Pelofsky|
|    laura|        bageant|           Steven|     Baskerville|
+---------+---------------+-----------------+----------------+
only showing top 2 rows



In [177]:
#Dropping the rows with null NAMELAST or visitee_namelast
df_non_null = df_req.filter(df_req.NAMELAST.isNotNull() & df_req.visitee_namelast.isNotNull())

In [135]:
df_null = df_req.filter(df_req.NAMELAST.isNull() | df_req.visitee_namelast.isNull())

In [136]:
df_null.count()

59255

In [137]:
from pyspark.sql.functions import lower, col

In [138]:
df_non_null.columns

['NAMEFIRST', 'NAMELAST', 'visitee_namefirst', 'visitee_namelast']

In [178]:
df_lower = df_non_null

In [179]:
cols = df_lower.columns

#Creating a for loop to convert all the columns to lower case
for col_name in cols:
        df_lower = df_lower.withColumn(col_name, lower(col(col_name)))

#Display data
df_lower.show(5)

+---------+---------------+-----------------+----------------+
|NAMEFIRST|       NAMELAST|visitee_namefirst|visitee_namelast|
+---------+---------------+-----------------+----------------+
|   hanene|tajouribessassi|             eric|        pelofsky|
|    laura|        bageant|           steven|     baskerville|
|     earl|       broemson|           steven|     baskerville|
|  william|    jackling jr|           steven|     baskerville|
|  richard|        mccrary|           steven|     baskerville|
+---------+---------------+-----------------+----------------+
only showing top 5 rows



In [139]:
#Converting all data and column names to lower case 

#creating a new dataframe
df_lower = df_non_null

#Getting all the column names
cols = df_lower.columns

#Creating a for loop to convert all the columns to lower case
for col_name in cols:
        df_lower = df_lower.withColumn(col_name, lower(col(col_name)))

#Display data
df_lower.show(5)

+---------+---------------+-----------------+----------------+
|NAMEFIRST|       NAMELAST|visitee_namefirst|visitee_namelast|
+---------+---------------+-----------------+----------------+
|   hanene|tajouribessassi|             eric|        pelofsky|
|    laura|        bageant|           steven|     baskerville|
|     earl|       broemson|           steven|     baskerville|
|  william|    jackling jr|           steven|     baskerville|
|  richard|        mccrary|           steven|     baskerville|
+---------+---------------+-----------------+----------------+
only showing top 5 rows



In [180]:
df_no_na = df_lower.na.drop(how='all')

In [140]:
####################### If a record is empty, then drop it
print(df_lower.filter((df_lower.NAMEFIRST != "") & (df_lower.NAMELAST != "") & (df_lower.visitee_namefirst != "") & (df_lower.visitee_namelast != "")).count())
print(df_lower.count())


908699
911249


In [181]:
df_lower.filter(df_lower.visitee_namefirst.isNull()).count()
df_lower.count()

911249

In [142]:
df.visitee_namelast.isNull()

AttributeError: 'DataFrame' object has no attribute 'visitee_namelast'

In [183]:
df = df_lower
n = 10

In [144]:
#(a) The 10 most frequent visitors to the White House.
#    (NAMELAST, NAMEFIRST)
df.select('NAMELAST', 'NAMEFIRST').groupby('NAMELAST', 'NAMEFIRST').\
count().orderBy('count', ascending=False).show(n)

+--------+---------+-----+
|NAMELAST|NAMEFIRST|count|
+--------+---------+-----+
| kidwell|   lauren|  222|
|  thomas| benjamin|  196|
|    haro|   steven|  183|
|  berner|katherine|  177|
|   grant|  patrick|  155|
|    haas|   jordan|  152|
|   garza|   steven|  127|
|   cohen|    mandy|  122|
|  martin|  kathryn|  122|
|   brown| jennifer|  117|
+--------+---------+-----+
only showing top 10 rows



In [184]:
df = df.withColumn('visitor', sf.concat(sf.col('NAMELAST'),sf.lit(', '), sf.col('NAMEFIRST')))

df.select('visitor').groupby('visitor').\
count().orderBy('count', ascending=False).show(n)

+-----------------+-----+
|          visitor|count|
+-----------------+-----+
|  kidwell, lauren|  222|
| thomas, benjamin|  196|
|     haro, steven|  183|
|berner, katherine|  177|
|   grant, patrick|  155|
|     haas, jordan|  152|
|    garza, steven|  127|
|  martin, kathryn|  122|
|     cohen, mandy|  122|
|  brown, jennifer|  117|
+-----------------+-----+
only showing top 10 rows



In [148]:
df.select('visitee_namelast', 'visitee_namefirst').groupby('visitee_namelast', 'visitee_namefirst').\
count().orderBy('count', ascending=False).show(n)

AnalysisException: "cannot resolve '`visitee_namelast, visitee_namefirst`' given input columns: [visitee, visitee_namefirst, NAMEFIRST, NAMELAST, visitor, visitee_namelast];;\n'Project ['visitee_namelast, visitee_namefirst]\n+- Project [NAMEFIRST#2013, NAMELAST#2018, visitee_namefirst#2023, visitee_namelast#2028, visitor#2110, concat(visitee_namelast#2028, , , visitee_namefirst#2023) AS visitee#2134]\n   +- Project [NAMEFIRST#2013, NAMELAST#2018, visitee_namefirst#2023, visitee_namelast#2028, concat(NAMELAST#2018, , , NAMEFIRST#2013) AS visitor#2110]\n      +- Project [NAMEFIRST#2013, NAMELAST#2018, visitee_namefirst#2023, lower(visitee_namelast#1831) AS visitee_namelast#2028]\n         +- Project [NAMEFIRST#2013, NAMELAST#2018, lower(visitee_namefirst#1832) AS visitee_namefirst#2023, visitee_namelast#1831]\n            +- Project [NAMEFIRST#2013, lower(NAMELAST#1812) AS NAMELAST#2018, visitee_namefirst#1832, visitee_namelast#1831]\n               +- Project [lower(NAMEFIRST#1813) AS N

In [187]:
#(b) The 10 most frequently visited people in 
#    the White House.
#    (visitee_namelast, visitee_namefirst) 

df = df.na.fill({'visitee_namefirst': '', 'NAMEFIRST': ''})

df = df.withColumn('visitee', sf.concat(sf.col('visitee_namelast'),sf.lit(', '), sf.col('visitee_namefirst')))


df.select('visitee').groupby('visitee').\
count().orderBy('count', ascending=False).show(10)

+--------------------+------+
|             visitee| count|
+--------------------+------+
|    office, visitors|430881|
|waves, visitorsof...| 44129|
|        bryant, ruth| 13970|
|       oneil, olivia| 13155|
|     thompson, jared| 11618|
|            /, potus| 10900|
|      burton, collin|  9672|
|      megan, matthew|  7944|
|     mayerson, asher|  6886|
| dessources, kalisha|  5289|
+--------------------+------+
only showing top 10 rows



In [188]:
#(c) The 10 most frequent visitor-visitee  combinations.
df.groupby('NAMELAST', 'NAMEFIRST','visitee_namelast', 'visitee_namefirst').\
count().orderBy('count', ascending=False).show(n)

+--------+---------+----------------+-----------------+-----+
|NAMELAST|NAMEFIRST|visitee_namelast|visitee_namefirst|count|
+--------+---------+----------------+-----------------+-----+
| kidwell|   lauren|        yudelson|             alex|  103|
|    haas|   jordan|        yudelson|             alex|   90|
|   grant|  patrick|        yudelson|             alex|   89|
|  thomas| benjamin|        yudelson|             alex|   89|
|    haro|   steven|        yudelson|             alex|   84|
|   cohen|    mandy|         lambrew|           jeanne|   84|
|  berner|katherine|        yudelson|             alex|   82|
|   roche|  shannon|        yudelson|             alex|   70|
|  urizar| jennifer|         johnson|            katie|   68|
|  martin|  kathryn|         lambrew|           jeanne|   61|
+--------+---------+----------------+-----------------+-----+
only showing top 10 rows



In [189]:
df = df.withColumn('visitor-visitee', sf.concat(sf.col('visitor'),sf.lit(' - '), sf.col('visitee')))

df.groupby('visitor-visitee').\
count().orderBy('count', ascending=False).show(n)

+--------------------+-----+
|     visitor-visitee|count|
+--------------------+-----+
|kidwell, lauren -...|  103|
|haas, jordan - yu...|   90|
|grant, patrick - ...|   89|
|thomas, benjamin ...|   89|
|cohen, mandy - la...|   84|
|haro, steven - yu...|   84|
|berner, katherine...|   82|
|roche, shannon - ...|   70|
|urizar, jennifer ...|   68|
|martin, kathryn -...|   61|
+--------------------+-----+
only showing top 10 rows



In [155]:
df.filter(df.visitee.isNull()).show()
#df_req.NAMELAST.isNotNull()

+---------+---------+-----------------+----------------+-----------------+-------+---------------+
|NAMEFIRST| NAMELAST|visitee_namefirst|visitee_namelast|          visitor|visitee|visitor-visitee|
+---------+---------+-----------------+----------------+-----------------+-------+---------------+
|    tanya|    matos|             null|         yushika|     matos, tanya|   null|           null|
|     ryan| mcdonald|             null|         yushika|   mcdonald, ryan|   null|           null|
|   steven|   mufson|             null|         yushika|   mufson, steven|   null|           null|
|   thomas|  prudden|             null|          marsha|  prudden, thomas|   null|           null|
|    bryan| strother|             null|          marsha|  strother, bryan|   null|           null|
|   robert|   custer|             null|          marsha|   custer, robert|   null|           null|
|    peter|    doocy|             null|         yushika|     doocy, peter|   null|           null|
|   andrew

In [191]:
df.groupby('visitor','visitee').\
count().orderBy('count', ascending=False).show(n)

+-----------------+---------------+-----+
|          visitor|        visitee|count|
+-----------------+---------------+-----+
|  kidwell, lauren| yudelson, alex|  103|
|     haas, jordan| yudelson, alex|   90|
|   grant, patrick| yudelson, alex|   89|
| thomas, benjamin| yudelson, alex|   89|
|     cohen, mandy|lambrew, jeanne|   84|
|     haro, steven| yudelson, alex|   84|
|berner, katherine| yudelson, alex|   82|
|   roche, shannon| yudelson, alex|   70|
| urizar, jennifer| johnson, katie|   68|
|  martin, kathryn|lambrew, jeanne|   61|
+-----------------+---------------+-----+
only showing top 10 rows



In [70]:
initial_records = df_req.count()
final_records = df.count()

print('Number of records dropped = ' + str(initial_records-final_records))
print('Number of records processed = ' + str(final_records))

Number of records dropped = 59255
Number of records processed = 911249
